In [1]:
from xml.etree import ElementTree
from xml.dom import minidom
import glob, os, sys, time
from shutil import copy
from xml.etree.ElementTree import Element, SubElement, Comment
from itertools import islice
import re

In [2]:
def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ElementTree.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

def gridXml(filename, header, top):
    reg = SubElement(top,'regular')
    reg.set('locationId','l.'+filename[0:7])
    child_1 = SubElement(reg, 'rows').text = str(header[1])
    child_2 = SubElement(reg, 'columns').text = str(header[0])
    child_3 = SubElement(reg, 'geoDatum').text = 'Rijks Driehoekstelsel'
    child_4 = SubElement(reg, 'firstCellCenter')
    child_4_sub = SubElement(child_4, 'x').text = str(header[2]+0.5)
    child_4_sub = SubElement(child_4, 'y').text = str(header[3]+header[1]-0.5)
    child_5 = SubElement(reg, 'xCellSize').text = str(header[4])
    child_6 = SubElement(reg, 'yCellSize').text = str(header[4])
    return top

def idMapXml(filename, top):
    map_ = SubElement(top,'map')
    map_.set('internalLocation','l.'+filename[0:7])
    map_.set('externalParameter',filename[0:7])
    map_.set('internalParameter','h.m')
    map_.set('externalLocation','ARC_INFO_LOC')
    return top

def LocationsXml(filename, top):
    map_ = SubElement(top,'location')
    map_.set('id','l.'+filename[0:7])
    map_.set('name','Baggervak '+filename[4:7])
    map_sub = SubElement(map_, 'x').text = str(0)
    map_sub = SubElement(map_, 'y').text = str(0)
    return top

def LocationSetsXml(filename, top, child):
    locId = SubElement(child,'locationId').text = 'l.'+filename[0:7]
    return top

non_decimal = re.compile(r'[^\d.,-]+')

In [3]:
asciiDataDir = r'D:\OMS_Waddenzee\trunk\fews'
rootDir = r'D:\OMS_Waddenzee\trunk\fews'
xmldir = os.path.join(rootDir, r'ImportInterim\geoxyz\bodempeilingen\XMLGenerated')

In [14]:
asciiFiles = []
asciiFile_fullPath = []
for root, dirs, files in os.walk(asciiDataDir):
    for file in files:
        if file.startswith('grid') and file.endswith('.asc'):
            asciiFiles.append(file)
            asciiFile_fullPath.append(os.path.join( os.path.abspath(root), file ))

asciiLocs = []
for file in asciiFiles:
    asciiLocs.append(file[0:7])

asciiLocsUnique = list(set(asciiLocs))
asciiLocsUnique.sort()

asciiFilesUnique = []
asciiFilesUnique_fullPath = []
for i in asciiLocsUnique:
    asciiFilesUnique.append(next(obj for obj in asciiFiles if obj[0:7]==i))
    asciiFilesUnique_fullPath.append(next(obj for obj in asciiFile_fullPath if obj[-26:-19]==i))

In [17]:
len(asciiFilesUnique_fullPath)

141

In [18]:
len(asciiFilesUnique)

141

In [12]:
asciiFile_fullPath[0][-26:-19]

'grid063'

In [19]:
# CREATE Grids.xml
# file to save
grids_xml = c = os.path.join(xmldir,'Grids.xml')
#grids_xml = r'D:\Projects\Pr\3317.20\BodempeilingScript_v4\xml\Grids.xml'

# start xml
top = Element('grids')
top.set('xmlns','http://www.wldelft.nl/fews')
top.set('xmlns:xsi','http://www.w3.org/2001/XMLSchema-instance')
top.set('xsi:schemaLocation','http://www.wldelft.nl/fews ../../../Schemas/grids.xsd')

for idx, grid in enumerate(asciiFilesUnique):
    init_file = asciiFilesUnique_fullPath[idx]
    with open(init_file, 'r') as f:
        head = list(islice(f, 6))
        #print (head)
        head_rp = non_decimal.sub('', str(head).replace('-',','))
        head_strp = [int(float(x.strip())) for x in head_rp.split(',') if x]
        top = gridXml(grid,head_strp, top)      

# save xml to file
with open(grids_xml, 'w') as the_file:
    the_file.write(prettify(top))

In [ ]:
# CREATE id.Surveyor.History.xml
# file to save
# id_Surveyor_History_xml = r'D:\Projects\Pr\3317.20\BodempeilingScript_v4\xml\id.Surveyor.History.xml'
id_Surveyor_History_xml = os.path.join(xmldir,'id.Surveyor.grids.xml')

# start xml
top = Element('idMap')
top.set('xmlns','http://www.wldelft.nl/fews')
top.set('xmlns:xsi','http://www.w3.org/2001/XMLSchema-instance')
top.set('xsi:schemaLocation','http://www.wldelft.nl/fews ../../../Schemas/idMap.xsd')
top.set('version','1.1')
for grid in asciiLocsUnique:
    top = idMapXml(grid, top)      

# save xml to file
with open(id_Surveyor_History_xml, 'w') as the_file:
    the_file.write(prettify(top))

In [ ]:
# CREATE Locations.xml
# file to save
# Locations_xml = r'D:\Projects\Pr\3317.20\BodempeilingScript_v4\xml\Locations.xml'
Locations_xml = os.path.join(xmldir,'Locations.xml')

# start xml
top = Element('locations')
top.set('version','1.1')
top.set('xmlns','http://www.wldelft.nl/fews')
top.set('xmlns:xsi','http://www.w3.org/2001/XMLSchema-instance')
top.set('xsi:schemaLocation','http://www.wldelft.nl/fews ../../../Schemas/idMap.xsd')
child = SubElement(top, 'geoDatum').text = 'Rijks Driehoekstelsel'

for grid in asciiLocsUnique:
    top = LocationsXml(grid, top)

# save xml to file
with open(Locations_xml, 'w') as the_file:
    the_file.write(prettify(top))

In [ ]:
# CREATE LocationSets.xml
# file to save
# LocationSets_xml = r'D:\Projects\Pr\3317.20\BodempeilingScript_v4\xml\LocationSets.xml'
LocationSets_xml = os.path.join(xmldir,'LocationSets.xml')
# start xml
top = Element('locationSets')
top.set('version','1.1')
top.set('xmlns','http://www.wldelft.nl/fews')
top.set('xmlns:xsi','http://www.w3.org/2001/XMLSchema-instance')
top.set('xsi:schemaLocation','http://www.wldelft.nl/fews ../../../Schemas/idMap.xsd')
child = SubElement(top, 'locationSet')
child.set('id','ls.baggervak.mosaic')
child.set('name','Baggervakken Mosaic')

for grid in asciiLocsUnique:
    top = LocationSetsXml(grid, top, child)

# save xml to file
with open(LocationSets_xml, 'w') as the_file:
    the_file.write(prettify(top))   